<a href="https://colab.research.google.com/github/jalevano/tfm_uoc_datascience/blob/main/02_sam2_evaluador_master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# =============================================================================
# SAM 2.0 EVALUADOR
# =============================================================================
# Proyecto: Evaluación Comparativa de Modelos de Segmentación de Personas
# Modelos: SAM 2.0 (Tiny, Small) - Extensible a Base Plus y Large
# =============================================================================

In [27]:
import os
from pathlib import Path

# Crear directorio de checkpoints
CHECKPOINTS_DIR = Path("/content/drive/MyDrive/TFM/sam2/checkpoints")
CHECKPOINTS_DIR.mkdir(parents=True, exist_ok=True)

# URLs oficiales de Meta (Facebook Research)
CHECKPOINTS = {
    'sam2_hiera_tiny.pt': 'https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_tiny.pt',
    'sam2_hiera_small.pt': 'https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_small.pt',
    'sam2_hiera_base_plus.pt': 'https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_base_plus.pt',
    'sam2_hiera_large.pt': 'https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_large.pt',
}

print("🔽 DESCARGANDO CHECKPOINTS SAM 2.0...")
print("="*80)

for filename, url in CHECKPOINTS.items():
    checkpoint_path = CHECKPOINTS_DIR / filename

    if checkpoint_path.exists():
        print(f"✅ {filename} ya existe (saltando descarga)")
    else:
        print(f"📥 Descargando {filename}...")
        print(f"   URL: {url}")

        # Descargar usando wget
        !wget -q --show-progress {url} -O {checkpoint_path}

        if checkpoint_path.exists():
            size_mb = checkpoint_path.stat().st_size / (1024 * 1024)
            print(f"   ✅ Descargado correctamente ({size_mb:.1f} MB)")
        else:
            print(f"   ❌ ERROR descargando {filename}")

print("\n" + "="*80)
print("✅ DESCARGA DE CHECKPOINTS COMPLETADA")
print(f"📁 Ubicación: {CHECKPOINTS_DIR}")

# Verificar archivos descargados
archivos = list(CHECKPOINTS_DIR.glob("*.pt"))
print(f"\n📊 Checkpoints disponibles: {len(archivos)}")
for archivo in archivos:
    size_mb = archivo.stat().st_size / (1024 * 1024)
    print(f"   • {archivo.name}: {size_mb:.1f} MB")

🔽 DESCARGANDO CHECKPOINTS SAM 2.0...
✅ sam2_hiera_tiny.pt ya existe (saltando descarga)
✅ sam2_hiera_small.pt ya existe (saltando descarga)
📥 Descargando sam2_hiera_base_plus.pt...
   URL: https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_base_plus.pt
/content/drive/MyDr 100%[===================>] 308.51M  35.2MB/s    in 7.8s    
   ✅ Descargado correctamente (308.5 MB)
📥 Descargando sam2_hiera_large.pt...
   URL: https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_large.pt
/content/drive/MyDr 100%[===================>] 856.35M  60.0MB/s    in 24s     
   ✅ Descargado correctamente (856.4 MB)

✅ DESCARGA DE CHECKPOINTS COMPLETADA
📁 Ubicación: /content/drive/MyDrive/TFM/sam2/checkpoints

📊 Checkpoints disponibles: 4
   • sam2_hiera_tiny.pt: 148.7 MB
   • sam2_hiera_small.pt: 175.8 MB
   • sam2_hiera_base_plus.pt: 308.5 MB
   • sam2_hiera_large.pt: 856.4 MB


In [2]:
# INSTALACIÓN DE DEPENDENCIAS
# =============================================================================

!pip install -q torch torchvision
!pip install -q opencv-python matplotlib Pillow
!pip install -q numpy scipy scikit-image
!pip install -q git+https://github.com/facebookresearch/segment-anything-2.git

# Librerías especializadas para análisis fotográfico (IGUAL QUE MASK2FORMER)
!pip install -q mahotas   # Análisis de texturas (Haralick features)
!pip install -q piexif    # Metadatos EXIF
!pip install -q shapely   # Análisis geométrico avanzado de polígonos

print("✅ Dependencias instaladas correctamente (incluyendo análisis fotográfico completo)")


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 76.1 MB/s eta 0:00:00
✅ Dependencias instaladas correctamente (incluyendo análisis fotográfico completo)


In [3]:
# =============================================================================
# IMPORTACIONES Y CONFIGURACIÓN INICIAL
# =============================================================================

import os
import sys
import json
import hashlib
import warnings
import gc
from pathlib import Path
from datetime import datetime
from dataclasses import dataclass, asdict
from typing import List, Dict, Any, Optional, Tuple
import time
import logging

import numpy as np
import torch
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap

# Librerías especializadas para análisis fotográfico
import mahotas
import piexif
from shapely.geometry import Polygon, Point
from shapely.validation import explain_validity
from skimage import feature, filters, measure, morphology, segmentation, transform
from skimage.color import rgb2gray, rgb2hsv, rgb2lab
import skimage

# SAM 2.0 imports
from sam2.build_sam import build_sam2
from sam2.automatic_mask_generator import SAM2AutomaticMaskGenerator

# Configuración para entorno Colab
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=False)
    IN_COLAB = True
    print("✅ Google Drive montado correctamente")
except:
    IN_COLAB = False
    print("⚠️ No estamos en Colab, continuando sin montar Drive...")

# Suprimir warnings
warnings.filterwarnings('ignore')

# Verificar GPU
if torch.cuda.is_available():
    print(f"✅ GPU disponible: {torch.cuda.get_device_name(0)}")
    print(f"   VRAM total: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    print("⚠️ GPU no disponible - SAM 2.0 requiere GPU para funcionar eficientemente")

Mounted at /content/drive
✅ Google Drive montado correctamente
✅ GPU disponible: Tesla T4
   VRAM total: 14.74 GB


In [28]:
# =============================================================================
# CELL 3: CONFIGURACIÓN Y ESTRUCTURAS DE DATOS
# =============================================================================

@dataclass
class ModeloSAMInfo:
    """Información completa de un modelo SAM 2.0"""
    nombre: str
    checkpoint: str
    config: str
    parametros_millones: int
    descripcion: str

    def obtener_nombre_sanitizado(self) -> str:
        """Obtiene nombre corto para archivos"""
        return self.nombre.replace('sam2_hiera_', '')

@dataclass
class ConfiguracionSAM:
    """Configuración de generación automática de máscaras"""
    nombre: str
    points_per_side: int
    pred_iou_thresh: float
    stability_score_thresh: float
    crop_n_layers: int
    crop_n_points_downscale_factor: int
    min_mask_region_area: int
    descripcion: str

@dataclass
class ConfigEvaluacionSAM:
    """Configuración centralizada del sistema de evaluación SAM 2.0"""

    # Rutas base
    BASE_PATH: Path = Path("/content/drive/MyDrive/TFM/sam2")
    DATASET_PATH: Path = BASE_PATH / "imagenes"
    CHECKPOINTS_PATH: Path = BASE_PATH / "checkpoints"

    # Parámetros de procesamiento
    MAX_SIZE_IMAGEN: int = 1024
    GUARDAR_VISUALIZACIONES: bool = True
    FORMATO_COCO_HABILITADO: bool = True

    # Modelos SAM 2.0 a evaluar (OPTIMIZADO PARA RECURSOS LIMITADOS)
    MODELOS: List[ModeloSAMInfo] = None

    # Configuraciones de generación de máscaras
    CONFIGURACIONES: Dict[str, ConfiguracionSAM] = None

    def __post_init__(self):
        """Inicializa modelos y configuraciones"""

        # MODELOS SAM 2.0 - Configuración LOW-COST
        self.MODELOS = [
            ModeloSAMInfo(
                nombre='sam2_hiera_tiny',
                checkpoint='sam2_hiera_tiny.pt',
                config='sam2_hiera_t.yaml',
                parametros_millones=39,
                descripcion='Modelo ultraligero para aplicaciones móviles y edge computing'
            ),
            ModeloSAMInfo(
                nombre='sam2_hiera_small',
                checkpoint='sam2_hiera_small.pt',
                config='sam2_hiera_s.yaml',
                parametros_millones=46,
                descripcion='Balance óptimo calidad/velocidad para producción'
            ),
             ModeloSAMInfo(
                 nombre='sam2_hiera_base_plus',
                 checkpoint='sam2_hiera_base_plus.pt',
                 config='sam2_hiera_b+.yaml',
                 parametros_millones=80,
                 descripcion='Modelo avanzado para producción de alta calidad'
             ),
             ModeloSAMInfo(
                 nombre='sam2_hiera_large',
                 checkpoint='sam2_hiera_large.pt',
                 config='sam2_hiera_l.yaml',
                 parametros_millones=224,
                 descripcion='Modelo de investigación - máxima calidad'
             ),
        ]

        self.CONFIGURACIONES = {
            'low_cost_tiny': ConfiguracionSAM(
                nombre='low_cost_tiny',
                points_per_side=16,  # 16x16 = 256 prompts (rápido)
                pred_iou_thresh=0.88,
                stability_score_thresh=0.92,
                crop_n_layers=0,  # Sin crops para ahorrar memoria
                crop_n_points_downscale_factor=2,
                min_mask_region_area=200,  # Filtrar máscaras pequeñas
                descripcion='Configuración ultra-rápida optimizada para Tiny'
            ),
            'balanced_small': ConfiguracionSAM(
                nombre='balanced_small',
                points_per_side=24,  # 24x24 = 576 prompts (balance)
                pred_iou_thresh=0.86,
                stability_score_thresh=0.92,
                crop_n_layers=1,
                crop_n_points_downscale_factor=2,
                min_mask_region_area=150,
                descripcion='Configuración balanceada para Small'
            ),
             'full_quality': ConfiguracionSAM(
                 nombre='full_quality',
                 points_per_side=32,  # 32x32 = 1024 prompts (completo)
                 pred_iou_thresh=0.86,
                 stability_score_thresh=0.92,
                 crop_n_layers=1,
                 crop_n_points_downscale_factor=2,
                 min_mask_region_area=100,
                 descripcion='Configuración completa para máxima calidad'
             ),
        }

    def validar_configuracion(self) -> bool:
        """Valida que la configuración sea correcta"""
        try:
            # Validar rutas
            if not self.DATASET_PATH.exists():
                print(f"❌ ERROR: Dataset no encontrado en {self.DATASET_PATH}")
                return False

            # Contar imágenes
            imagenes = list(self.DATASET_PATH.glob('*.jpg')) + \
                      list(self.DATASET_PATH.glob('*.jpeg')) + \
                      list(self.DATASET_PATH.glob('*.png'))

            if len(imagenes) == 0:
                print(f"❌ ERROR: No se encontraron imágenes en {self.DATASET_PATH}")
                return False

            print(f"✅ Dataset: {len(imagenes)} imágenes encontradas")

            # Crear directorio de checkpoints si no existe
            self.CHECKPOINTS_PATH.mkdir(parents=True, exist_ok=True)

            return True

        except Exception as e:
            print(f"❌ ERROR en validación: {str(e)}")
            return False

    def obtener_resumen_configuracion(self) -> Dict[str, Any]:
        """Obtiene resumen de la configuración"""
        imagenes = list(self.DATASET_PATH.glob('*.jpg')) + \
                  list(self.DATASET_PATH.glob('*.jpeg')) + \
                  list(self.DATASET_PATH.glob('*.png'))

        return {
            'total_modelos': len(self.MODELOS),
            'total_configuraciones': len(self.CONFIGURACIONES),
            'total_combinaciones': len(self.MODELOS) * len(self.CONFIGURACIONES),
            'total_imagenes': len(imagenes),
            'dataset_path': str(self.DATASET_PATH),
            'visualizaciones_habilitadas': self.GUARDAR_VISUALIZACIONES,
            'formato_coco_habilitado': self.FORMATO_COCO_HABILITADO,
        }

In [6]:
# =============================================================================
# SISTEMA DE LOGGING
# =============================================================================
class LoggerManager:
    """Gestión centralizada de logs del sistema"""

    def __init__(self, directorio_logs: Path):
        self.directorio_logs = directorio_logs
        self.directorio_logs.mkdir(parents=True, exist_ok=True)
        self.loggers: Dict[str, logging.Logger] = {}
        self.timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    def crear_logger(self, nombre: str) -> logging.Logger:
        """Crea un logger específico para un componente"""
        if nombre in self.loggers:
            return self.loggers[nombre]

        logger = logging.getLogger(f"SAM2_{nombre}_{self.timestamp}")
        logger.setLevel(logging.INFO)
        logger.handlers = []

        # Handler para archivo
        archivo_log = self.directorio_logs / f"{nombre}_{self.timestamp}.log"
        file_handler = logging.FileHandler(archivo_log, encoding='utf-8')
        file_handler.setLevel(logging.INFO)

        # Formato
        formatter = logging.Formatter(
            '%(asctime)s - %(name)s - %(levelname)s - %(message)s',
            datefmt='%Y-%m-%d %H:%M:%S'
        )
        file_handler.setFormatter(formatter)
        logger.addHandler(file_handler)

        # Handler para consola (solo errores)
        console_handler = logging.StreamHandler()
        console_handler.setLevel(logging.ERROR)
        console_handler.setFormatter(formatter)
        logger.addHandler(console_handler)

        self.loggers[nombre] = logger
        return logger

In [7]:
# =============================================================================
# UTILIDADES
# =============================================================================

class Utils:
    """Utilidades generales del sistema"""

    @staticmethod
    def limpiar_memoria():
        """Limpia memoria GPU y RAM"""
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

    @staticmethod
    def preparar_imagen(ruta: str, max_size: int = 1024) -> np.ndarray:
        """Prepara imagen para procesamiento con SAM"""
        try:
            imagen = cv2.imread(ruta)
            if imagen is None:
                raise ValueError(f"No se pudo cargar la imagen: {ruta}")

            imagen = cv2.cvtColor(imagen, cv2.COLOR_BGR2RGB)

            # Redimensionar si es necesario
            h, w = imagen.shape[:2]
            if max(h, w) > max_size:
                scale = max_size / max(h, w)
                new_w, new_h = int(w * scale), int(h * scale)
                imagen = cv2.resize(imagen, (new_w, new_h), interpolation=cv2.INTER_AREA)

            return imagen

        except Exception as e:
            raise ValueError(f"Error cargando imagen {ruta}: {str(e)}")

    @staticmethod
    def calcular_hash_imagen(ruta: str) -> str:
        """Calcula hash MD5 para identificación única de imagen"""
        try:
            with open(ruta, 'rb') as f:
                return hashlib.md5(f.read()).hexdigest()[:12]
        except Exception:
            return "hash_error"

    @staticmethod
    def guardar_json(datos: Any, archivo: Path, indent: int = 2) -> None:
        """Guarda datos en formato JSON con manejo de errores"""
        try:
            # Convertir numpy types a tipos nativos de Python
            def convert_types(obj):
                if isinstance(obj, np.integer):
                    return int(obj)
                elif isinstance(obj, np.floating):
                    return float(obj)
                elif isinstance(obj, np.ndarray):
                    return obj.tolist()
                elif isinstance(obj, dict):
                    return {k: convert_types(v) for k, v in obj.items()}
                elif isinstance(obj, list):
                    return [convert_types(item) for item in obj]
                return obj

            datos_convertidos = convert_types(datos)

            with open(archivo, 'w', encoding='utf-8') as f:
                json.dump(datos_convertidos, f, indent=indent, ensure_ascii=False, default=str)
        except Exception as e:
            raise IOError(f"Error guardando JSON en {archivo}: {str(e)}")

    @staticmethod
    def crear_nombre_archivo(modelo_info: ModeloSAMInfo, config_nombre: str,
                           timestamp: str) -> str:
        """Crea nombres de archivo descriptivos y únicos"""
        modelo_nombre = modelo_info.obtener_nombre_sanitizado()
        return f"sam2_{modelo_nombre}_{config_nombre}_{timestamp}.json"

In [8]:
# =============================================================================
# EXTRACTOR DE CARACTERÍSTICAS AVANZADAS
# =============================================================================

class ExtractorCaracteristicasAvanzado:
    """
    Extractor de características fotográficas.
    """

    def __init__(self):
        self.logger = None

    def set_logger(self, logger: logging.Logger):
        """Asigna logger para el extractor"""
        self.logger = logger

    def analizar_imagen_completa(self, imagen: Image.Image, ruta: str) -> Dict[str, Any]:
        """Análisis completo usando librerías especializadas"""
        img_array = np.array(imagen)
        img_gray = rgb2gray(img_array)
        img_hsv = rgb2hsv(img_array)
        img_lab = rgb2lab(img_array)

        img_gray_uint8 = (img_gray * 255).astype(np.uint8)

        caracteristicas = {
            'metadatos_basicos': self._extraer_metadatos_basicos(imagen, ruta),
            'color_y_paleta': self._analizar_color_avanzado(img_array, img_hsv, img_lab),
            'texturas_mahotas': self._extraer_texturas_mahotas(img_gray_uint8),
            'texturas_skimage': self._extraer_texturas_skimage(img_gray),
            'caracteristicas_geometricas': self._analizar_geometria_avanzada(img_gray_uint8),
            'multiscale_features': self._extraer_multiscale_features(img_gray),
            'propiedades_regionales': self._analizar_propiedades_regionales(img_gray),
            'descriptores_locales': self._extraer_descriptores_locales(img_gray_uint8)
        }

        return caracteristicas

    def _extraer_metadatos_basicos(self, imagen: Image.Image, ruta: str) -> Dict:
        """Metadatos básicos de la imagen"""
        w, h = imagen.size
        return {
            'dimensiones': {'ancho': w, 'alto': h},
            'aspecto_ratio': round(w / h, 3),
            'megapixeles': round((w * h) / 1000000, 2),
            'orientacion': 'horizontal' if w > h else 'vertical' if h > w else 'cuadrada',
            'formato': ruta.split('.')[-1].lower() if '.' in ruta else 'desconocido'
        }

    def _analizar_color_avanzado(self, img_rgb: np.ndarray, img_hsv: np.ndarray,
                                img_lab: np.ndarray) -> Dict:
        """Análisis avanzado de color en múltiples espacios"""
        return {
            'estadisticas_rgb': {
                'mean': img_rgb.mean(axis=(0, 1)).tolist(),
                'std': img_rgb.std(axis=(0, 1)).tolist(),
                'min': img_rgb.min(axis=(0, 1)).tolist(),
                'max': img_rgb.max(axis=(0, 1)).tolist()
            },
            'hsv_distribucion': {
                'hue_medio': float(img_hsv[:, :, 0].mean()),
                'saturacion_media': float(img_hsv[:, :, 1].mean()),
                'valor_medio': float(img_hsv[:, :, 2].mean())
            },
            'lab_luminancia': {
                'L_medio': float(img_lab[:, :, 0].mean()),
                'a_medio': float(img_lab[:, :, 1].mean()),
                'b_medio': float(img_lab[:, :, 2].mean())
            }
        }

    def _extraer_texturas_mahotas(self, img_gray: np.ndarray) -> Dict:
        """Características de textura usando Mahotas"""
        try:
            haralick = mahotas.features.haralick(img_gray, return_mean=True)
            return {
                'haralick_mean': haralick.tolist() if haralick is not None else []
            }
        except:
            return {'haralick_mean': []}

    def _extraer_texturas_skimage(self, img_gray: np.ndarray) -> Dict:
        """Características de textura usando scikit-image"""
        try:
            # GLCM
            glcm = feature.graycomatrix(
                (img_gray * 255).astype(np.uint8),
                distances=[1],
                angles=[0, np.pi/4, np.pi/2, 3*np.pi/4],
                levels=256,
                symmetric=True,
                normed=True
            )

            return {
                'contrast': float(feature.graycoprops(glcm, 'contrast').mean()),
                'dissimilarity': float(feature.graycoprops(glcm, 'dissimilarity').mean()),
                'homogeneity': float(feature.graycoprops(glcm, 'homogeneity').mean()),
                'energy': float(feature.graycoprops(glcm, 'energy').mean()),
                'correlation': float(feature.graycoprops(glcm, 'correlation').mean())
            }
        except:
            return {}

    def _analizar_geometria_avanzada(self, img_gray: np.ndarray) -> Dict:
        """Análisis geométrico avanzado"""
        try:
            edges = feature.canny(img_gray)
            return {
                'edge_density': float(edges.sum() / edges.size),
                'shape_index': float(img_gray.std() / img_gray.mean()) if img_gray.mean() > 0 else 0
            }
        except:
            return {}

    def _extraer_multiscale_features(self, img_gray: np.ndarray) -> Dict:
        """Características multi-escala"""
        try:
            pyramid = list(skimage.transform.pyramid_gaussian(img_gray, max_layer=3))
            return {
                'num_scales': len(pyramid),
                'scale_variance': [float(level.std()) for level in pyramid]
            }
        except:
            return {}

    def _analizar_propiedades_regionales(self, img_gray: np.ndarray) -> Dict:
        """Propiedades de regiones"""
        try:
            threshold = filters.threshold_otsu(img_gray)
            binary = img_gray > threshold
            labeled = measure.label(binary)
            regions = measure.regionprops(labeled)

            if regions:
                areas = [r.area for r in regions]
                return {
                    'num_regions': len(regions),
                    'mean_region_area': float(np.mean(areas)),
                    'std_region_area': float(np.std(areas))
                }
            return {'num_regions': 0}
        except:
            return {}

    def _extraer_descriptores_locales(self, img_gray: np.ndarray) -> Dict:
        """Descriptores locales (keypoints)"""
        try:
            # Detectar corners
            corners = feature.corner_peaks(
                feature.corner_harris(img_gray),
                min_distance=5
            )

            return {
                'num_corners': len(corners),
                'corner_density': float(len(corners) / img_gray.size)
            }
        except:
            return {}

In [10]:
# =============================================================================
# ANALIZADOR DE MÁSCARAS CON SHAPELY
# =============================================================================

class MaskAnalyzer:
    """
    Analizador avanzado de máscaras usando Shapely.
    """

    def __init__(self, epsilon_factor: float = 0.005):
        self.epsilon_factor = epsilon_factor

    def analizar_mascara_completa(self, mask: np.ndarray,
                                  imagen_shape: Tuple[int, int]) -> Dict[str, Any]:
        """
        Análisis completo de una máscara individual

        Args:
            mask: Máscara binaria (numpy array 2D)
            imagen_shape: (altura, anchura) de la imagen original

        Returns:
            Dict con análisis geométrico completo
        """
        resultado = {
            'caracteristicas_geometricas': {},
            'shapely_geometry': {},
            'caracteristicas_forma': {},
            'caracteristicas_contextuales': {}
        }

        try:
            mask_binary = mask.astype(np.uint8)

            # Análisis geométrico básico (scikit-image)
            resultado['caracteristicas_geometricas'] = self._extraer_geometricas_basicas(
                mask_binary, imagen_shape
            )

            # Análisis avanzado con Shapely
            resultado['shapely_geometry'] = self._analizar_con_shapely(mask_binary)

            # Características de forma avanzadas
            resultado['caracteristicas_forma'] = self._extraer_caracteristicas_forma(
                mask_binary
            )

            # Características contextuales (posición en imagen)
            resultado['caracteristicas_contextuales'] = self._extraer_contextuales(
                mask_binary, imagen_shape
            )

        except Exception as e:
            resultado['error'] = str(e)

        return resultado

    def _extraer_geometricas_basicas(self, mask: np.ndarray,
                                    imagen_shape: Tuple[int, int]) -> Dict[str, float]:
        """Extrae características geométricas básicas usando scikit-image"""
        try:
            props = measure.regionprops(mask.astype(int))[0] if np.any(mask) else None

            if props is None:
                return self._geometricas_vacias()

            total_pixels = imagen_shape[0] * imagen_shape[1]

            return {
                'area_pixels': float(props.area),
                'area_percentage': float(props.area / total_pixels * 100),
                'perimeter': float(props.perimeter),
                'compactness': float(4 * np.pi * props.area / (props.perimeter ** 2)) if props.perimeter > 0 else 0.0,
                'aspect_ratio': float(props.major_axis_length / props.minor_axis_length) if props.minor_axis_length > 0 else 0.0,
                'orientation_angle': float(np.degrees(props.orientation)),
                'centroid_y': float(props.centroid[0]),
                'centroid_x': float(props.centroid[1]),
                'solidity': float(props.solidity),
                'extent': float(props.extent),
                'eccentricity': float(props.eccentricity),
                'major_axis_length': float(props.major_axis_length),
                'minor_axis_length': float(props.minor_axis_length)
            }
        except:
            return self._geometricas_vacias()

    def _analizar_con_shapely(self, mask: np.ndarray) -> Dict[str, Any]:
        """Análisis geométrico avanzado usando Shapely"""
        try:
            # Encontrar contornos
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            if not contours:
                return self._shapely_vacio()

            # Contorno principal
            contour = max(contours, key=cv2.contourArea)
            epsilon = self.epsilon_factor * cv2.arcLength(contour, True)
            contour_simplified = cv2.approxPolyDP(contour, epsilon, True)

            if len(contour_simplified) < 3:
                return self._shapely_vacio()

            # Crear polígono Shapely
            coords = [(point[0][0], point[0][1]) for point in contour_simplified]
            polygon = Polygon(coords)

            # Validar polígono
            is_valid = polygon.is_valid
            validity_msg = explain_validity(polygon) if not is_valid else "Valid"

            if not is_valid:
                # Intentar reparar
                polygon = polygon.buffer(0)
                is_valid = polygon.is_valid

            # Calcular métricas geométricas
            convex_hull = polygon.convex_hull

            return {
                'polygon_valido': bool(is_valid),
                'validity_message': validity_msg,
                'area_shapely': float(polygon.area),
                'perimeter_shapely': float(polygon.length),
                'num_vertices': len(coords),
                'convex_hull_area': float(convex_hull.area),
                'convexity_ratio': float(polygon.area / convex_hull.area) if convex_hull.area > 0 else 0.0,
                'is_simple': bool(polygon.is_simple),
                'bounds': list(polygon.bounds)  # (minx, miny, maxx, maxy)
            }
        except Exception as e:
            return {**self._shapely_vacio(), 'error': str(e)}

    def _extraer_caracteristicas_forma(self, mask: np.ndarray) -> Dict[str, float]:
        """Características de forma avanzadas"""
        try:
            # Momentos de Hu (invariantes a escala, rotación, traslación)
            moments = cv2.moments(mask)
            hu_moments = cv2.HuMoments(moments).flatten()

            # Fourier descriptors (aproximación simple)
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
            if contours:
                contour = max(contours, key=cv2.contourArea)

                # Curvatura aproximada
                perimeter = cv2.arcLength(contour, True)
                area = cv2.contourArea(contour)

                return {
                    'hu_moment_1': float(hu_moments[0]),
                    'hu_moment_2': float(hu_moments[1]),
                    'hu_moment_3': float(hu_moments[2]),
                    'circularity': float(4 * np.pi * area / (perimeter ** 2)) if perimeter > 0 else 0.0,
                    'roughness_factor': float(perimeter / (2 * np.sqrt(np.pi * area))) if area > 0 else 0.0
                }

            return {}
        except:
            return {}

    def _extraer_contextuales(self, mask: np.ndarray,
                             imagen_shape: Tuple[int, int]) -> Dict[str, Any]:
        """Características contextuales (posición relativa en imagen)"""
        try:
            h, w = imagen_shape
            props = measure.regionprops(mask.astype(int))[0] if np.any(mask) else None

            if props is None:
                return {}

            cy, cx = props.centroid

            # Posición relativa normalizada
            rel_x = cx / w
            rel_y = cy / h

            # Cuadrante (1-9, tipo numpad)
            if rel_y < 0.33:
                quadrant_y = 'superior'
            elif rel_y < 0.66:
                quadrant_y = 'centro'
            else:
                quadrant_y = 'inferior'

            if rel_x < 0.33:
                quadrant_x = 'izquierda'
            elif rel_x < 0.66:
                quadrant_x = 'centro'
            else:
                quadrant_x = 'derecha'

            quadrant = f"{quadrant_y}_{quadrant_x}"

            # Distancia al centro
            center_dist = np.sqrt((rel_x - 0.5)**2 + (rel_y - 0.5)**2)

            return {
                'centroid_normalized_x': float(rel_x),
                'centroid_normalized_y': float(rel_y),
                'quadrant': quadrant,
                'distance_to_center': float(center_dist),
                'is_centered': bool(center_dist < 0.2)  # Dentro del 20% central
            }
        except:
            return {}

    def _geometricas_vacias(self) -> Dict[str, float]:
        """Retorna estructura vacía para características geométricas"""
        return {
            'area_pixels': 0.0,
            'area_percentage': 0.0,
            'perimeter': 0.0,
            'compactness': 0.0,
            'aspect_ratio': 0.0,
            'orientation_angle': 0.0,
            'centroid_y': 0.0,
            'centroid_x': 0.0,
            'solidity': 0.0,
            'extent': 0.0,
            'eccentricity': 0.0,
            'major_axis_length': 0.0,
            'minor_axis_length': 0.0
        }

    def _shapely_vacio(self) -> Dict[str, Any]:
        """Retorna estructura vacía para análisis Shapely"""
        return {
            'polygon_valido': False,
            'validity_message': 'No polygon created',
            'area_shapely': 0.0,
            'perimeter_shapely': 0.0,
            'num_vertices': 0,
            'convex_hull_area': 0.0,
            'convexity_ratio': 0.0,
            'is_simple': False,
            'bounds': [0.0, 0.0, 0.0, 0.0]
        }

In [11]:
class FiltradorPersonas:
    """Filtra máscaras que probablemente corresponden a personas"""

    def __init__(self, logger: logging.Logger = None):
        self.logger = logger or logging.getLogger(__name__)

        # Parámetros heurísticos para detección de personas en fotografía
        self.MIN_AREA_RATIO = 0.05  # Mínimo 5% de la imagen
        self.MAX_AREA_RATIO = 0.90  # Máximo 90% de la imagen
        self.MIN_ASPECT_RATIO = 0.3  # Ancho/Alto mínimo
        self.MAX_ASPECT_RATIO = 1.5  # Ancho/Alto máximo (personas son más altas)
        self.MIN_PREDICTED_IOU = 0.85  # Confianza mínima de SAM
        self.PREFERRED_CENTER_DISTANCE = 0.3  # Distancia al centro preferida

    def filtrar_mascaras_personas(self, mascaras: List[Dict],
                                  imagen_shape: Tuple[int, int]) -> List[Dict]:
        """
        Filtra máscaras que probablemente son personas en fotografía de retrato

        Args:
            mascaras: Lista de máscaras generadas por SAM
            imagen_shape: (altura, anchura) de la imagen

        Returns:
            Lista de máscaras filtradas y ordenadas por probabilidad de ser persona
        """
        if not mascaras:
            return []

        h, w = imagen_shape
        area_total = h * w
        centro_x, centro_y = w / 2, h / 2

        mascaras_filtradas = []

        for mask_data in mascaras:
            segmentacion = mask_data['segmentation']
            bbox = mask_data['bbox']  # [x, y, width, height]
            predicted_iou = mask_data.get('predicted_iou', 0)

            # Calcular características
            area = np.sum(segmentacion)
            area_ratio = area / area_total

            x, y, mask_w, mask_h = bbox
            aspect_ratio = mask_w / mask_h if mask_h > 0 else 0

            # Centro de la máscara
            mask_centro_x = x + mask_w / 2
            mask_centro_y = y + mask_h / 2

            # Distancia al centro (normalizada)
            dist_centro = np.sqrt(
                ((mask_centro_x - centro_x) / w) ** 2 +
                ((mask_centro_y - centro_y) / h) ** 2
            )

            # CRITERIOS DE FILTRADO
            criterios_cumplidos = 0
            max_criterios = 5

            # 1. Tamaño apropiado
            if self.MIN_AREA_RATIO <= area_ratio <= self.MAX_AREA_RATIO:
                criterios_cumplidos += 1

            # 2. Aspect ratio de persona (más alto que ancho)
            if self.MIN_ASPECT_RATIO <= aspect_ratio <= self.MAX_ASPECT_RATIO:
                criterios_cumplidos += 1

            # 3. Confianza alta de SAM
            if predicted_iou >= self.MIN_PREDICTED_IOU:
                criterios_cumplidos += 1

            # 4. Posición central (típico en retratos)
            if dist_centro <= self.PREFERRED_CENTER_DISTANCE:
                criterios_cumplidos += 1

            # 5. Compacidad (personas son objetos compactos)
            perimetro_aprox = 2 * (mask_w + mask_h)
            compacidad = (4 * np.pi * area) / (perimetro_aprox ** 2) if perimetro_aprox > 0 else 0
            if compacidad > 0.2:  # Umbral de compacidad
                criterios_cumplidos += 1

            # Calcular score de confianza
            confidence_score = criterios_cumplidos / max_criterios

            # FILTRO: Al menos 3 de 5 criterios
            if criterios_cumplidos >= 3:
                mask_data['person_confidence'] = confidence_score
                mask_data['criteria_met'] = criterios_cumplidos
                mask_data['area_ratio'] = float(area_ratio)
                mask_data['aspect_ratio'] = float(aspect_ratio)
                mask_data['center_distance'] = float(dist_centro)
                mascaras_filtradas.append(mask_data)

        # Ordenar por confianza (mayor primero)
        mascaras_filtradas.sort(key=lambda x: x['person_confidence'], reverse=True)

        self.logger.info(
            f"Filtrado: {len(mascaras)} máscaras → {len(mascaras_filtradas)} personas candidatas"
        )

        return mascaras_filtradas

In [12]:
# =============================================================================
# GENERADOR DE MÁSCARAS SAM 2.0
# =============================================================================

class GeneradorMascarasSAM:
    """Genera máscaras automáticas usando SAM 2.0"""

    def __init__(self, modelo_info: ModeloSAMInfo, config_sam: ConfiguracionSAM,
                 checkpoints_path: Path, logger: logging.Logger):
        self.modelo_info = modelo_info
        self.config_sam = config_sam
        self.checkpoints_path = checkpoints_path
        self.logger = logger

        self.modelo = None
        self.mask_generator = None
        self.filtrador = FiltradorPersonas(logger)

    def cargar_modelo(self):
        """Carga el modelo SAM 2.0"""
        try:
            self.logger.info(f"Cargando modelo {self.modelo_info.nombre}...")

            # Ruta al checkpoint
            checkpoint_path = self.checkpoints_path / self.modelo_info.checkpoint

            if not checkpoint_path.exists():
                raise FileNotFoundError(
                    f"Checkpoint no encontrado: {checkpoint_path}\n"
                    f"Descarga los checkpoints desde: https://github.com/facebookresearch/segment-anything-2#model-checkpoints"
                )

            # Construir modelo
            device = "cuda" if torch.cuda.is_available() else "cpu"
            self.modelo = build_sam2(
                config_file=self.modelo_info.config,
                ckpt_path=str(checkpoint_path),
                device=device
            )

            # Crear generador automático
            self.mask_generator = SAM2AutomaticMaskGenerator(
                model=self.modelo,
                points_per_side=self.config_sam.points_per_side,
                pred_iou_thresh=self.config_sam.pred_iou_thresh,
                stability_score_thresh=self.config_sam.stability_score_thresh,
                crop_n_layers=self.config_sam.crop_n_layers,
                crop_n_points_downscale_factor=self.config_sam.crop_n_points_downscale_factor,
                min_mask_region_area=self.config_sam.min_mask_region_area,
            )

            self.logger.info(f"✅ Modelo {self.modelo_info.nombre} cargado correctamente")

        except Exception as e:
            self.logger.error(f"Error cargando modelo: {str(e)}")
            raise

    def generar_mascaras(self, imagen: np.ndarray) -> Dict[str, Any]:
        """
        Genera máscaras automáticas y filtra personas

        Returns:
            Dict con máscaras totales, personas detectadas y metadatos
        """
        try:
            inicio = time.time()

            # Generar todas las máscaras
            self.logger.info("Generando máscaras automáticas...")
            mascaras_todas = self.mask_generator.generate(imagen)

            tiempo_generacion = (time.time() - inicio) * 1000

            # Filtrar máscaras de personas
            mascaras_personas = self.filtrador.filtrar_mascaras_personas(
                mascaras_todas,
                imagen.shape[:2]
            )

            resultado = {
                'total_mascaras_generadas': len(mascaras_todas),
                'personas_detectadas': len(mascaras_personas),
                'mascaras_personas': mascaras_personas,
                'mascaras_todas': mascaras_todas,  # Guardar todas por si acaso
                'tiempo_generacion_ms': tiempo_generacion,
                'imagen_shape': imagen.shape[:2],
            }

            self.logger.info(
                f"✅ Generación completa: {len(mascaras_todas)} máscaras → "
                f"{len(mascaras_personas)} personas en {tiempo_generacion:.1f}ms"
            )

            return resultado

        except Exception as e:
            self.logger.error(f"Error generando máscaras: {str(e)}")
            raise

    def liberar_memoria(self):
        """Libera memoria del modelo"""
        del self.modelo
        del self.mask_generator
        self.modelo = None
        self.mask_generator = None
        Utils.limpiar_memoria()
        self.logger.info("Memoria liberada")

In [13]:
# =============================================================================
# GESTOR DE MÁSCARAS (CON ANÁLISIS SHAPELY)
# =============================================================================

class GestorMascaras:
    """Gestiona el almacenamiento de máscaras individuales CON análisis geométrico"""

    def __init__(self, directorio_salida: Path, logger: logging.Logger):
        self.directorio_salida = directorio_salida
        self.logger = logger
        self.directorio_mascaras = directorio_salida / "masks"
        self.directorio_mascaras.mkdir(parents=True, exist_ok=True)

        # Analizador de máscaras con Shapely
        self.mask_analyzer = MaskAnalyzer()

    def guardar_mascaras(self, mascaras_personas: List[Dict],
                        nombre_archivo: str, hash_imagen: str,
                        imagen_shape: Tuple[int, int]) -> Dict[str, Any]:
        """
        Guarda máscaras individuales CON análisis geométrico completo

        Returns:
            Metadatos de las máscaras guardadas (incluyendo análisis Shapely)
        """
        try:
            if not mascaras_personas:
                return {'total_guardadas': 0, 'archivos': []}

            archivos_guardados = []

            for idx, mask_data in enumerate(mascaras_personas):
                # Análisis geométrico completo de la máscara
                analisis_geometrico = self.mask_analyzer.analizar_mascara_completa(
                    mask_data['segmentation'],
                    imagen_shape
                )

                # Nombre del archivo
                nombre_base = Path(nombre_archivo).stem
                archivo_mask = self.directorio_mascaras / f"{nombre_base}_mask_{idx:03d}.npz"

                # Guardar máscara y metadatos completos
                np.savez_compressed(
                    archivo_mask,
                    segmentation=mask_data['segmentation'].astype(np.uint8),
                    bbox=np.array(mask_data['bbox']),
                    area=mask_data['area'],
                    predicted_iou=mask_data.get('predicted_iou', 0),
                    stability_score=mask_data.get('stability_score', 0),
                    person_confidence=mask_data.get('person_confidence', 0),
                    criteria_met=mask_data.get('criteria_met', 0),
                )

                # Metadatos para JSON (incluye análisis Shapely)
                archivos_guardados.append({
                    'archivo': archivo_mask.name,
                    'indice': idx,
                    'area': int(mask_data['area']),
                    'confianza_persona': float(mask_data.get('person_confidence', 0)),
                    'bbox': mask_data['bbox'],
                    'analisis_geometrico': analisis_geometrico
                })

            metadatos = {
                'total_guardadas': len(archivos_guardados),
                'archivos': archivos_guardados,
                'directorio': str(self.directorio_mascaras)
            }

            self.logger.info(f"✅ Guardadas {len(archivos_guardados)} máscaras con análisis geométrico")

            return metadatos

        except Exception as e:
            self.logger.error(f"Error guardando máscaras: {str(e)}")
            return {'total_guardadas': 0, 'archivos': [], 'error': str(e)}

In [14]:
# =============================================================================
# GENERADOR DE VISUALIZACIONES
# =============================================================================

class GeneradorVisualizacionesSAM:
    """Genera visualizaciones comparativas de resultados SAM"""

    def __init__(self, directorio_salida: Path, logger: logging.Logger = None):
        self.directorio_salida = directorio_salida
        self.directorio_salida.mkdir(parents=True, exist_ok=True)
        self.logger = logger or logging.getLogger(__name__)

    def generar_visualizacion_completa(self, imagen_original: np.ndarray,
                                      nombre_archivo: str,
                                      resultado_generacion: Dict,
                                      modelo_nombre: str = "SAM2") -> Optional[str]:
        """
        Genera visualización completa: original + todas las máscaras + solo personas
        """
        try:
            mascaras_todas = resultado_generacion.get('mascaras_todas', [])
            mascaras_personas = resultado_generacion.get('mascaras_personas', [])

            if not mascaras_todas:
                self.logger.warning("No hay máscaras para visualizar")
                return None

            # Crear figura con 3 subplots
            fig, axes = plt.subplots(1, 3, figsize=(20, 7))
            fig.suptitle(
                f'{modelo_nombre} - {nombre_archivo}\n'
                f'Total: {len(mascaras_todas)} máscaras | Personas: {len(mascaras_personas)}',
                fontsize=16, fontweight='bold'
            )

            # 1. Imagen original
            axes[0].imshow(imagen_original)
            axes[0].set_title('Imagen Original', fontsize=14)
            axes[0].axis('off')

            # 2. Todas las máscaras
            self._visualizar_mascaras(axes[1], imagen_original, mascaras_todas,
                                     f'Todas las Máscaras ({len(mascaras_todas)})')

            # 3. Solo personas
            self._visualizar_mascaras(axes[2], imagen_original, mascaras_personas,
                                     f'Personas Detectadas ({len(mascaras_personas)})',
                                     mostrar_confianza=True)

            # Guardar
            nombre_vis = Path(nombre_archivo).stem + f'_{modelo_nombre}_completo.png'
            ruta_salida = self.directorio_salida / nombre_vis
            plt.tight_layout()
            plt.savefig(ruta_salida, dpi=150, bbox_inches='tight')
            plt.close()

            self.logger.info(f"✅ Visualización guardada: {ruta_salida.name}")

            return str(ruta_salida)

        except Exception as e:
            self.logger.error(f"Error generando visualización: {str(e)}")
            plt.close('all')
            return None

    def _visualizar_mascaras(self, ax, imagen: np.ndarray, mascaras: List[Dict],
                            titulo: str, mostrar_confianza: bool = False):
        """Visualiza máscaras en un eje"""
        ax.imshow(imagen)

        if not mascaras:
            ax.set_title(f'{titulo} - Vacío', fontsize=12)
            ax.axis('off')
            return

        # Crear overlay con colores aleatorios
        overlay = np.zeros_like(imagen, dtype=np.float32)

        for mask_data in mascaras:
            seg = mask_data['segmentation']
            color = np.random.random(3)

            # Aplicar máscara con transparencia
            for c in range(3):
                overlay[:, :, c] += seg * color[c] * 0.6

        # Normalizar y mostrar
        overlay = np.clip(overlay, 0, 1)
        ax.imshow(overlay, alpha=0.5)

        # Añadir bounding boxes si es visualización de personas
        if mostrar_confianza:
            for mask_data in mascaras:
                bbox = mask_data['bbox']
                confianza = mask_data.get('person_confidence', 0)

                rect = mpatches.Rectangle(
                    (bbox[0], bbox[1]), bbox[2], bbox[3],
                    fill=False, edgecolor='lime', linewidth=2
                )
                ax.add_patch(rect)

                # Etiqueta con confianza
                ax.text(
                    bbox[0], bbox[1] - 5,
                    f'{confianza:.2f}',
                    color='lime', fontsize=10, fontweight='bold',
                    bbox=dict(facecolor='black', alpha=0.7, pad=2)
                )

        ax.set_title(titulo, fontsize=12)
        ax.axis('off')

In [16]:
# =============================================================================
# PROCESADOR DE RESULTADOS (CON ANÁLISIS FOTOGRÁFICO)
# =============================================================================

class ProcesadorResultados:
    """Procesa y almacena resultados de evaluación CON análisis fotográfico completo"""

    def __init__(self, directorio_salida: Path, logger_manager: LoggerManager):
        self.directorio_salida = directorio_salida
        self.logger_manager = logger_manager
        self.logger = logger_manager.crear_logger("procesador")

        # Extractor de características fotográficas (IGUAL QUE MASK2FORMER)
        self.extractor = ExtractorCaracteristicasAvanzado()
        self.extractor.set_logger(self.logger)

        # Componentes
        self.gestor_mascaras = GestorMascaras(directorio_salida, self.logger)
        self.generador_vis = GeneradorVisualizacionesSAM(
            directorio_salida / "visualizaciones",
            self.logger
        )

        # Contadores
        self.imagenes_procesadas = 0
        self.imagenes_exitosas = 0
        self.tiempo_total_procesamiento = 0

    def procesar_imagen(self, ruta_imagen: str,
                       generador: GeneradorMascarasSAM,
                       modelo_nombre: str, config_nombre: str,
                       guardar_visualizacion: bool = True) -> Optional[Dict[str, Any]]:
        """Procesa una imagen y guarda resultados CON análisis fotográfico completo"""

        inicio_procesamiento = time.time()
        nombre_archivo = os.path.basename(ruta_imagen)

        try:
            # Preparar imagen
            imagen_np = Utils.preparar_imagen(ruta_imagen)
            imagen_pil = Image.fromarray(imagen_np)
            hash_imagen = Utils.calcular_hash_imagen(ruta_imagen)

            self.logger.info(f"Extrayendo características fotográficas de {nombre_archivo}...")
            caracteristicas_avanzadas = self.extractor.analizar_imagen_completa(
                imagen_pil,
                ruta_imagen
            )

            # Información completa de la imagen
            info_imagen = {
                'archivo': nombre_archivo,
                'ruta_completa': ruta_imagen,
                'hash_md5': hash_imagen,
                'resolucion': {
                    'altura': imagen_np.shape[0],
                    'anchura': imagen_np.shape[1],
                    'canales': imagen_np.shape[2] if len(imagen_np.shape) > 2 else 1
                },
                'caracteristicas_avanzadas': caracteristicas_avanzadas
            }

            # Generar máscaras
            resultado_mascaras = generador.generar_mascaras(imagen_np)

            # Guardar máscaras individuales con análisis geométrico
            metadatos_mascaras = self.gestor_mascaras.guardar_mascaras(
                resultado_mascaras['mascaras_personas'],
                nombre_archivo,
                hash_imagen,
                imagen_np.shape[:2]
            )

            # Generar visualización
            ruta_visualizacion = None
            if guardar_visualizacion:
                ruta_visualizacion = self.generador_vis.generar_visualizacion_completa(
                    imagen_np, nombre_archivo, resultado_mascaras,
                    modelo_nombre=f"{modelo_nombre}_{config_nombre}"
                )

            # Construir resultado completo
            tiempo_procesamiento = (time.time() - inicio_procesamiento) * 1000

            resultado = {
                'timestamp': datetime.now().isoformat(),
                'modelo': modelo_nombre,
                'configuracion': config_nombre,
                'imagen': info_imagen,
                'deteccion': {
                    'total_mascaras_generadas': resultado_mascaras['total_mascaras_generadas'],
                    'personas_detectadas': resultado_mascaras['personas_detectadas'],
                    'tiempo_generacion_ms': resultado_mascaras['tiempo_generacion_ms'],
                    'mascaras_guardadas': metadatos_mascaras,
                },
                'visualizacion': {
                    'archivo': Path(ruta_visualizacion).name if ruta_visualizacion else None,
                    'ruta': str(ruta_visualizacion) if ruta_visualizacion else None
                },
                'rendimiento': {
                    'tiempo_total_procesamiento_ms': tiempo_procesamiento,
                },
                'exito': True
            }

            # Actualizar contadores
            self.imagenes_procesadas += 1
            self.imagenes_exitosas += 1
            self.tiempo_total_procesamiento += tiempo_procesamiento

            self.logger.info(
                f"✅ {nombre_archivo} - {resultado_mascaras['personas_detectadas']} personas "
                f"({tiempo_procesamiento:.1f}ms)"
            )

            return resultado

        except Exception as e:
            self.logger.error(f"❌ Error procesando {nombre_archivo}: {str(e)}")

            self.imagenes_procesadas += 1

            return {
                'timestamp': datetime.now().isoformat(),
                'modelo': modelo_nombre,
                'imagen': {'archivo': nombre_archivo},
                'error': str(e),
                'exito': False
            }

In [18]:
# =============================================================================
# EVALUADOR PRINCIPAL SAM 2.0
# =============================================================================

class EvaluadorSAM2:
    """Evaluador principal de modelos SAM 2.0"""

    def __init__(self, config: ConfigEvaluacionSAM):
        self.config = config

        # Crear directorio de ejecución con timestamp
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        self.directorio_ejecucion = config.BASE_PATH / "resultados" / f"{timestamp}_evaluacion_sam2"
        self.directorio_ejecucion.mkdir(parents=True, exist_ok=True)

        # Sistema de logging
        self.logger_manager = LoggerManager(self.directorio_ejecucion / "logs")
        self.logger = self.logger_manager.crear_logger("evaluador_principal")

        self.logger.info("="*80)
        self.logger.info("EVALUADOR SAM 2.0 INICIALIZADO")
        self.logger.info(f"Directorio de ejecución: {self.directorio_ejecucion}")
        self.logger.info("="*80)

        # Estadísticas
        self.estadisticas_globales = {
            'timestamp_inicio': timestamp,
            'modelos_evaluados': [],
            'imagenes_procesadas': 0,
            'imagenes_exitosas': 0,
        }

    def ejecutar_evaluacion_completa(self) -> Dict[str, List[str]]:
        """Ejecuta evaluación completa de todos los modelos y configuraciones"""

        archivos_resultado = {}

        # Obtener lista de imágenes
        imagenes = self._obtener_imagenes_dataset()
        total_imagenes = len(imagenes)

        self.logger.info(f"Iniciando evaluación con {total_imagenes} imágenes")

        # Iterar sobre modelos
        for modelo_info in self.config.MODELOS:
            modelo_nombre = modelo_info.obtener_nombre_sanitizado()

            print(f"\n{'='*100}")
            print(f"🤖 EVALUANDO MODELO: {modelo_info.nombre}")
            print(f"   Parámetros: {modelo_info.parametros_millones}M")
            print(f"   Descripción: {modelo_info.descripcion}")
            print(f"{'='*100}\n")

            # Seleccionar configuración apropiada para el modelo
            if 'tiny' in modelo_info.nombre:
                config_usar = self.config.CONFIGURACIONES['low_cost_tiny']
            else:
                config_usar = self.config.CONFIGURACIONES['balanced_small']

            archivos_modelo = self._evaluar_modelo(
                modelo_info,
                config_usar,
                imagenes
            )

            archivos_resultado[modelo_nombre] = archivos_modelo

            # Liberar memoria entre modelos
            Utils.limpiar_memoria()

            print(f"\n✅ Modelo {modelo_nombre} completado\n")

        # Guardar estadísticas globales
        self._guardar_estadisticas_globales()

        return archivos_resultado

    def _evaluar_modelo(self, modelo_info: ModeloSAMInfo,
                       config_sam: ConfiguracionSAM,
                       imagenes: List[Path]) -> List[str]:
        """Evalúa un modelo específico con una configuración"""

        modelo_nombre = modelo_info.obtener_nombre_sanitizado()
        config_nombre = config_sam.nombre

        # Crear directorio para este modelo
        dir_modelo = self.directorio_ejecucion / modelo_nombre
        dir_json = dir_modelo / "json"
        dir_json.mkdir(parents=True, exist_ok=True)

        # Inicializar componentes
        generador = GeneradorMascarasSAM(
            modelo_info, config_sam,
            self.config.CHECKPOINTS_PATH,
            self.logger_manager.crear_logger(f"generador_{modelo_nombre}")
        )

        procesador = ProcesadorResultados(
            dir_modelo,
            self.logger_manager
        )

        # Cargar modelo
        try:
            generador.cargar_modelo()
        except Exception as e:
            self.logger.error(f"Error cargando modelo {modelo_nombre}: {str(e)}")
            return []

        # Procesar imágenes
        resultados = []
        archivos_guardados = []

        for idx, ruta_imagen in enumerate(imagenes, 1):
            print(f"   [{idx}/{len(imagenes)}] Procesando: {ruta_imagen.name}...", end=' ')

            resultado = procesador.procesar_imagen(
                str(ruta_imagen),
                generador,
                modelo_nombre,
                config_nombre,
                guardar_visualizacion=self.config.GUARDAR_VISUALIZACIONES
            )

            if resultado:
                resultados.append(resultado)

                if resultado.get('exito'):
                    print(f"✅ {resultado['deteccion']['personas_detectadas']} personas")
                else:
                    print(f"❌ Error")

            # Limpiar memoria cada imagen (importante para Colab)
            Utils.limpiar_memoria()

        # Guardar resultados en JSON
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        nombre_json = Utils.crear_nombre_archivo(modelo_info, config_nombre, timestamp)
        archivo_json = dir_json / nombre_json

        resumen = {
            'metadata': {
                'modelo': asdict(modelo_info),
                'configuracion': asdict(config_sam),
                'timestamp': timestamp,
                'total_imagenes': len(imagenes),
                'imagenes_exitosas': procesador.imagenes_exitosas,
            },
            'resultados': resultados,
            'estadisticas': {
                'imagenes_procesadas': procesador.imagenes_procesadas,
                'imagenes_exitosas': procesador.imagenes_exitosas,
                'tiempo_total_ms': procesador.tiempo_total_procesamiento,
            }
        }

        Utils.guardar_json(resumen, archivo_json)
        archivos_guardados.append(str(archivo_json))

        # Actualizar estadísticas globales
        self.estadisticas_globales['modelos_evaluados'].append(modelo_nombre)
        self.estadisticas_globales['imagenes_procesadas'] += procesador.imagenes_procesadas
        self.estadisticas_globales['imagenes_exitosas'] += procesador.imagenes_exitosas

        # Liberar modelo
        generador.liberar_memoria()

        return archivos_guardados

    def _obtener_imagenes_dataset(self) -> List[Path]:
        """Obtiene lista de imágenes del dataset"""
        extensiones = ['*.jpg', '*.jpeg', '*.png', '*.JPG', '*.JPEG', '*.PNG']
        imagenes = []
        for ext in extensiones:
            imagenes.extend(self.config.DATASET_PATH.glob(ext))

        imagenes.sort()
        return imagenes

    def _guardar_estadisticas_globales(self):
        """Guarda estadísticas globales de la ejecución"""
        archivo_stats = self.directorio_ejecucion / "estadisticas_globales.json"
        Utils.guardar_json(self.estadisticas_globales, archivo_stats)
        self.logger.info(f"Estadísticas globales guardadas en {archivo_stats}")

    def obtener_estadisticas_ejecucion(self) -> Dict[str, Any]:
        """Obtiene estadísticas de la ejecución"""
        return {
            'directorio_ejecucion': str(self.directorio_ejecucion),
            'modelos_evaluados': len(self.estadisticas_globales['modelos_evaluados']),
            'imagenes_procesadas': self.estadisticas_globales['imagenes_procesadas'],
            'imagenes_exitosas': self.estadisticas_globales['imagenes_exitosas'],
        }

In [22]:
# ============================================================================
# FUNCIÓN PRINCIPAL
# ============================================================================
def main():
    """
      Función principal única del sistema de evaluación SAM 2.0
      Ejecuta automáticamente todos los modelos configurados
    """
    try:
        # Configuración inicial
        config = ConfigEvaluacionSAM()

        print(f"\n{'='*100}")
        print(f"SISTEMA DE EVALUACIÓN SAM 2.0 - OPTIMIZADO PARA RECURSOS LIMITADOS")
        print(f"{'='*100}")

        print(f"\nConfiguración de evaluación:")
        print(f"   Modelos SAM 2.0: {len(config.MODELOS)}")
        for i, modelo in enumerate(config.MODELOS):
            print(f"      [{i}] {modelo.nombre} ({modelo.parametros_millones}M params)")

        print(f"   Configuraciones: {len(config.CONFIGURACIONES)}")
        for nombre, cfg in config.CONFIGURACIONES.items():
            print(f"      {nombre}: {cfg.descripcion}")

        # Validar configuración
        if not config.validar_configuracion():
            print("❌ ERROR: Configuración inválida. Verificar paths y parámetros.")
            return

        print("✅ Configuración validada correctamente")

        # Crear evaluador
        evaluador = EvaluadorSAM2(config)

        # Mostrar resumen
        resumen_config = config.obtener_resumen_configuracion()
        print(f"\nRESUMEN DE CONFIGURACIÓN:")
        print(f"   Total combinaciones: {resumen_config['total_combinaciones']}")
        print(f"   Total imágenes: {resumen_config['total_imagenes']}")
        print(f"   Dataset: {resumen_config['dataset_path']}")
        print(f"   Visualizaciones: {'✓ Habilitadas' if resumen_config['visualizaciones_habilitadas'] else '✗ Deshabilitadas'}")

        # Ejecutar evaluación completa
        print(f"\n🚀 INICIANDO EVALUACIÓN AUTOMÁTICA COMPLETA...")
        archivos_resultado = evaluador.ejecutar_evaluacion_completa()

        # Obtener estadísticas finales
        estadisticas = evaluador.obtener_estadisticas_ejecucion()

        # Resumen final
        print(f"\n{'='*100}")
        print(f"✅ EVALUACIÓN COMPLETA FINALIZADA EXITOSAMENTE")
        print(f"{'='*100}")

        print(f"📁 Directorio de resultados: {estadisticas['directorio_ejecucion']}")
        print(f"📊 Imágenes procesadas: {estadisticas['imagenes_procesadas']}")
        print(f"✅ Imágenes exitosas: {estadisticas['imagenes_exitosas']}")
        print(f"🤖 Modelos evaluados: {estadisticas['modelos_evaluados']}")

        print(f"\n📂 RESUMEN POR MODELO:")
        for modelo, archivos in archivos_resultado.items():
            print(f"   {modelo}: {len(archivos)} evaluaciones completadas")

        print(f"\n{'='*100}")
        print("🎉 ¡EVALUACIÓN SAM 2.0 COMPLETADA!")
        print(f"{'='*100}\n")

    except Exception as e:
        print(f"\n❌ ERROR CRÍTICO: {str(e)}")
        import traceback
        traceback.print_exc()

In [29]:
# =============================================================================
# EJECUTAR EVALUACIÓN
# =============================================================================

if __name__ == "__main__":
    main()

INFO:SAM2_evaluador_principal_20251005_192422:================================================================================
INFO:SAM2_evaluador_principal_20251005_192422:EVALUADOR SAM 2.0 INICIALIZADO
INFO:SAM2_evaluador_principal_20251005_192422:Directorio de ejecución: /content/drive/MyDrive/TFM/sam2/resultados/20251005_192422_evaluacion_sam2
INFO:SAM2_evaluador_principal_20251005_192422:================================================================================
INFO:SAM2_evaluador_principal_20251005_192422:Iniciando evaluación con 6 imágenes
INFO:SAM2_generador_tiny_20251005_192422:Cargando modelo sam2_hiera_tiny...



SISTEMA DE EVALUACIÓN SAM 2.0 - OPTIMIZADO PARA RECURSOS LIMITADOS

Configuración de evaluación:
   Modelos SAM 2.0: 4
      [0] sam2_hiera_tiny (39M params)
      [1] sam2_hiera_small (46M params)
      [2] sam2_hiera_base_plus (80M params)
      [3] sam2_hiera_large (224M params)
   Configuraciones: 3
      low_cost_tiny: Configuración ultra-rápida optimizada para Tiny
      balanced_small: Configuración balanceada para Small
      full_quality: Configuración completa para máxima calidad
✅ Dataset: 6 imágenes encontradas
✅ Configuración validada correctamente

RESUMEN DE CONFIGURACIÓN:
   Total combinaciones: 12
   Total imágenes: 6
   Dataset: /content/drive/MyDrive/TFM/sam2/imagenes
   Visualizaciones: ✓ Habilitadas

🚀 INICIANDO EVALUACIÓN AUTOMÁTICA COMPLETA...

🤖 EVALUANDO MODELO: sam2_hiera_tiny
   Parámetros: 39M
   Descripción: Modelo ultraligero para aplicaciones móviles y edge computing



INFO:SAM2_generador_tiny_20251005_192422:✅ Modelo sam2_hiera_tiny cargado correctamente


   [1/6] Procesando: 1.jpg... 

INFO:SAM2_procesador_20251005_192422:Extrayendo características fotográficas de 1.jpg...
INFO:SAM2_generador_tiny_20251005_192422:Generando máscaras automáticas...
INFO:SAM2_generador_tiny_20251005_192422:Filtrado: 5 máscaras → 3 personas candidatas
INFO:SAM2_generador_tiny_20251005_192422:✅ Generación completa: 5 máscaras → 3 personas en 1527.7ms
INFO:SAM2_procesador_20251005_192422:✅ Guardadas 3 máscaras con análisis geométrico
INFO:SAM2_procesador_20251005_192422:✅ Visualización guardada: 1_tiny_low_cost_tiny_completo.png
INFO:SAM2_procesador_20251005_192422:✅ 1.jpg - 3 personas (5813.8ms)


...

✅ 28 personas
   [6/6] Procesando: 7.jpg... 

INFO:SAM2_procesador_20251005_192422:Extrayendo características fotográficas de 7.jpg...
INFO:SAM2_generador_large_20251005_192422:Generando máscaras automáticas...
INFO:SAM2_generador_large_20251005_192422:Filtrado: 25 máscaras → 16 personas candidatas
INFO:SAM2_generador_large_20251005_192422:✅ Generación completa: 25 máscaras → 16 personas en 9081.4ms
INFO:SAM2_procesador_20251005_192422:✅ Guardadas 16 máscaras con análisis geométrico
INFO:SAM2_procesador_20251005_192422:✅ Visualización guardada: 7_large_balanced_small_completo.png
INFO:SAM2_procesador_20251005_192422:✅ 7.jpg - 16 personas (13110.6ms)


✅ 16 personas


INFO:SAM2_generador_large_20251005_192422:Memoria liberada
INFO:SAM2_evaluador_principal_20251005_192422:Estadísticas globales guardadas en /content/drive/MyDrive/TFM/sam2/resultados/20251005_192422_evaluacion_sam2/estadisticas_globales.json



✅ Modelo large completado


✅ EVALUACIÓN COMPLETA FINALIZADA EXITOSAMENTE
📁 Directorio de resultados: /content/drive/MyDrive/TFM/sam2/resultados/20251005_192422_evaluacion_sam2
📊 Imágenes procesadas: 24
✅ Imágenes exitosas: 24
🤖 Modelos evaluados: 4

📂 RESUMEN POR MODELO:
   tiny: 1 evaluaciones completadas
   small: 1 evaluaciones completadas
   base_plus: 1 evaluaciones completadas
   large: 1 evaluaciones completadas

🎉 ¡EVALUACIÓN SAM 2.0 COMPLETADA!

